In [1]:
import time            
import re            
import os    
import sys  
import codecs  
import shutil  
import numpy as np  
import matplotlib  
import scipy  
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn import feature_extraction    
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
# #結巴 匯入自己的詞典
# jieba.set_dictionary('./dict/jieba_dict.txt.big')

#停止詞字典stopwordset
stopwordset = set()
with open('./dic/stopwords.txt', 'r', encoding='utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

In [4]:
#一个商品
review = open('../CN_20171130/pre/review.csv',encoding='utf-8').read().split('\n')
review[0]

'"头可动，模型也很逼真"|商品到达得很快，很不错~||||||||'

In [5]:
#一条评论
Reviews=[]
for rev in review:
    a = rev.split('|')
    Reviews += a 
Reviews = Reviews[:-1]
len(Reviews)

6290

In [6]:
# 断词
import jieba  
import jieba.analyse  
corpus=[]
for review in Reviews:
#     if review !='':
    words=jieba.cut(review, cut_all=False)
    pincut=" ".join([word for word in words if word not in stopwordset and '\u4e00' <= word and word <= '\u9fff'])
    corpus.append(pincut)
# corpus =corpus[:-1]   
len(corpus)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/hz/s0m8h80s4bb47624crxjd6h40000gn/T/jieba.cache
Loading model cost 1.063 seconds.
Prefix dict has been built succesfully.


6290

In [9]:
#title
asins = open('../CN_20171130/pre/ASIN.csv').read().split('\n')
ASINs=[]
for item in asins:
    a = item.split('\t')
    ASINs += a
# ASINs =ASINs[:-1]   
len(ASINs)

6290

In [11]:
#genres
categorys = open('../CN_20171130/pre/categories.csv').read().split('\n')

Category=[]
for cat in categorys:
    a = cat.split('\t')
    Category += a
# Category = Category[:-1]
len(Category)

6290

In [13]:
#ranks
ranks = open('../CN_20171130/pre/ranks.csv').read().split('\n')
ranks[0] = ranks[0].replace('\ufeff','')
Rank =[]
for rank in ranks:
    a = rank.split('\t')
    Rank += a
    
Ranks =[float(i) for i in Rank]  
Ranks[0]
len(Ranks)

6290

## Tf-idf and document similarity

In [14]:
#将评论中的词语转换为词频矩阵 矩阵元素a[i][j] 表示j词在第i类评论下的词频  
vectorizer = CountVectorizer()

In [15]:
#该类会统计每个词语的tf-idf权值  
transformer = TfidfTransformer() 

In [16]:
tf = vectorizer.fit_transform(corpus)

In [17]:
#第一个fit_transform是计算tf-idf 第二个fit_transform是将文本转为词频矩阵  
tfidf = transformer.fit_transform(tf)

In [18]:
#获取词袋模型中的前10个词语    
word = vectorizer.get_feature_names()
word[:10]

['一丁点', '一上', '一下子', '一下子把', '一不小心', '一不留神', '一个个', '一个劲', '一个半', '一个多']

In [19]:
# 词袋模型中共有几个词语
len(word)

5360

In [20]:
#将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重  
weight = tfidf.toarray() 
weight

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [21]:
weight.shape

(6290, 5360)

In [22]:
#词频矩阵
tfidframe = pd.DataFrame(np.round(weight, 2))

In [23]:
tfidframe[:10]

,0,1,2,3,4,5,6,7,8,9,...,5350,5351,5352,5353,5354,5355,5356,5357,5358,5359
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## cosine similarity

In [24]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(tfidf)
dist = 1 - similarity

In [25]:
print(dist)

[[ -2.22044605e-16   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   9.04035014e-01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   1.00000000e+00
    1.00000000e+00   1.00000000e+00]
 ..., 
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,  -2.22044605e-16
    9.77831781e-01   8.81090336e-01]
 [  1.00000000e+00   1.00000000e+00   1.00000000e+00 ...,   9.77831781e-01
   -6.66133815e-16   1.00000000e+00]
 [  1.00000000e+00   9.04035014e-01   1.00000000e+00 ...,   8.81090336e-01
    1.00000000e+00   0.00000000e+00]]


In [26]:
similarity_df = pd.DataFrame(similarity)

In [27]:
similarity_df[:10]

,0,1,2,3,4,5,6,7,8,9,...,6280,6281,6282,6283,6284,6285,6286,6287,6288,6289
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.029923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095965
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


# K-means clustering

In [28]:
from sklearn.cluster import KMeans  
num_clusters = 10
km = KMeans(n_clusters=10)   
%time s = km.fit(weight)  
clusters = km.labels_.tolist()
print(s)

CPU times: user 26.8 s, sys: 3.21 s, total: 30 s
Wall time: 24.6 s
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)


In [29]:
# from scipy.spatial.distance import cdist
# X = weight #在weight矩阵中元素w[i][j]表示j词在i类文本中的tf-idf权重  
# K = range(1, 100)
# meandistortions = []
# for k in K:
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(X)
#     # 求kmeans的成本函数值
#     meandistortions.append(sum(np.min(cdist(X, kmeans.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

# plt.figure()
# plt.grid(True)
# plt1 = plt.subplot(2,1,1)
# # 画样本点
# plt1.plot(X[:,0],X[:,1],'k.');
# plt2 = plt.subplot(2,1,2)
# # 画成本函数值曲线
# plt2.plot(K, meandistortions, 'bx-')
# plt.show()

In [30]:
from sklearn.externals import joblib
# 注释语句用来存储你的模型
# 因为我已经从 pickle 载入过模型了
 
joblib.dump(km,  'doc_cluster.pkl')
 
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [31]:
toys = { 'title': ASINs, 'rank': Ranks, 'review': Reviews, 'cluster': clusters, 'category': Category }
 
frame = pd.DataFrame(toys , index = [clusters] , columns = ['rank', 'title', 'cluster', 'category'])

In [32]:
# frame['cluster'].value_counts() #number of review per cluster (clusters from 0 to 10)

In [33]:
# km.inertia_

In [34]:
# grouped = frame['rank'].groupby(frame['cluster']) # 为了凝聚（aggregation），由聚类分类。
 
# grouped.mean() # 每个聚类的平均排名（0 到 5）

In [35]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
 
# for i in range(num_clusters):
#     print("Cluster %d titles:" % i, end='')
#     for title in frame.ix[i]['title'].values.tolist():
#         print(' %s,' % title, end='')
#     print() # 空行
#     print() # 空行

In [36]:
# from __future__ import print_function
 
# print("Top terms per cluster:")
# print()
# # 按离质心的距离排列聚类中心，由近到远
# order_centroids = km.cluster_centers_.argsort()[:, ::-1] 
# # print(order_centroids)
# for i in range(num_clusters):
#     print("Cluster %d:" % i, end='')
#     time.sleep(1)
#     for ind in order_centroids[i, :50]: # 每个聚类选 10 个词
#         print(' %s' % word[ind]  , end=',')
#     print() # 空行
#     print() # 空行

## LDA

In [37]:
from sklearn.decomposition import LatentDirichletAllocation

In [38]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [39]:
%time lda.fit(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 35.2 s, sys: 258 ms, total: 35.5 s
Wall time: 36.3 s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [40]:
%time lda_matrix = lda.fit_transform(tf) #tf 为向量化后的语料词集

/usr/local/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


CPU times: user 36.4 s, sys: 338 ms, total: 36.8 s
Wall time: 38 s


In [41]:
lda_matrix #表示每一個文檔屬於每一個聚類的概率

array([[ 0.02500027,  0.025     ,  0.025     , ...,  0.025     ,
         0.025     ,  0.77499973],
       [ 0.020011  ,  0.02      ,  0.02000212, ...,  0.0200003 ,
         0.02000436,  0.57462934],
       [ 0.1       ,  0.1       ,  0.1       , ...,  0.1       ,
         0.1       ,  0.1       ],
       ..., 
       [ 0.00769264,  0.00769231,  0.77692142, ...,  0.00769247,
         0.00769231,  0.16153815],
       [ 0.00588308,  0.00588235,  0.00588432, ...,  0.07251232,
         0.00588289,  0.00588235],
       [ 0.84998988,  0.01666667,  0.0166741 , ...,  0.01666694,
         0.01666791,  0.01666718]])

In [42]:
#https://www.pixpo.net/education/0IJdxJRZ.html
features = pd.DataFrame(lda_matrix, columns=['T1', 'T2', 'T3','T4','T5','T6','T7','T8','T9','T10'])

In [43]:
features[:5]

,T1,T2,T3,T4,T5,T6,T7,T8,T9,T10
0,0.025000,0.025,0.025000,0.025,0.025,0.025,0.025000,0.025,0.025000,0.775000
1,0.020011,0.020,0.020002,0.020,0.020,0.020,0.265353,0.020,0.020004,0.574629
2,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000
3,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000
4,0.100000,0.100,0.100000,0.100,0.100,0.100,0.100000,0.100,0.100000,0.100000


In [44]:
%time km_lda =km.fit_transform(features)

CPU times: user 161 ms, sys: 10.3 ms, total: 171 ms
Wall time: 166 ms


In [45]:
cluster_labels = km.labels_
cluster_labels = pd.DataFrame(cluster_labels, columns=['ClusterLabel'])

In [46]:
ldaframe = pd.DataFrame({ 'title': ASINs, 'rank': Ranks, 'review': Reviews,'category': Category})
ldaframe = ldaframe[['title','rank','category','review']]
ldaClusterframe = pd.concat([ldaframe, cluster_labels], axis=1)
ldaClusterframe[:5]

,title,rank,category,review,ClusterLabel
0,"""B06WWD7SMQ_1",5.0,1,"""头可动，模型也很逼真""",6
1,B06WWD7SMQ_2,5.0,1,商品到达得很快，很不错~,6
2,B06WWD7SMQ_3,0.0,1,,0
3,B06WWD7SMQ_4,0.0,1,,0
4,B06WWD7SMQ_5,0.0,1,,0


In [47]:
ldaClusterframe['ClusterLabel'].value_counts()

0    2620
1    1061
3     539
4     441
2     352
5     344
6     308
8     235
9     206
7     184
Name: ClusterLabel, dtype: int64

In [48]:
Rankgrouped = ldaClusterframe['rank'].groupby(ldaClusterframe['ClusterLabel']) # 为了凝聚（aggregation），由聚类分类。
 
Rankgrouped.mean() # 每个聚类的平均排名（0 到 5）

ClusterLabel
0    0.643511
1    4.634307
2    4.585227
3    4.688312
4    4.360544
5    4.148256
6    4.074675
7    4.000000
8    4.255319
9    4.233010
Name: rank, dtype: float64

In [49]:
#排序
ldaClusterframe.sort_values(by = 'ClusterLabel')[:10]

,title,rank,category,review,ClusterLabel
5137,B01B7GAGWM_8,0.0,9,,0
4936,B07219PL55_7,0.0,9,,0
4935,B07219PL55_6,0.0,9,,0
4934,B07219PL55_5,0.0,9,,0
4933,B07219PL55_4,0.0,9,,0
2586,B006ZWJJO0_7,4.0,5,造型生动，细节处理较为细致。跑动轻巧速度，很不错。小配件真心要小心，易脱落。,0
4932,B07219PL55_3,0.0,9,,0
4927,B00VM16OYM_8,5.0,9,安全性高，前面轮子大，遇到障碍停下来，不会撞着；安装很简单，实际颜色没有照片鲜艳。至于转弯实...,0
2598,B01BX4F79K_9,5.0,5,需要在开阔场地玩，飞的很高，很不错,0
4920,"""B00VM16OYM_1",5.0,9,"""东西很好非常满意。做工扎实。快递也很给力，头天定的第二天就送到了。唯一不足是，价格很高。淘...",0


In [50]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
#     print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        
        
    print() # 空行
    print() # 空行

 不错, 喜欢, 质量, 做工, 玩具, 孩子, 宝宝, 小孩, 价格, 儿子, 味道, 好玩, 手感, 设计, 图片, 东西, 活动, 异味, 开心, 精致, 小车, 爱玩, 牌子, 想象, 划算, 精细, 超级, 粗糙, 购买, 车子,

 难度, 车厢, 回家, 系列, 客服, 照片, 实物, 小男孩, 有时候, 想要, 卡扣, 超过, 情况, 哇哇, 几种, 一摸, 太软, 半小时, 用时, 放在, 太好了, 绒毛, 稀少, 上深, 紫霞, 车子, 组合, 木屑, 人员, 吃惊,

 孩子, 喜欢, 宝宝, 一岁, 便宜, 质量, 朋友, 可爱, 适合, 玩儿, 一套, 精致, 清晰, 天天, 还会, 颜色, 玩偶, 送给, 第一个, 很快, 三岁, 兴趣, 摸摸, 窒息, 看不懂, 感觉, 小孩, 厚实, 可惜, 侄子,

 适合, 柔软, 简单, 小孩子, 毛绒, 舒服, 介绍, 漂亮, 袋子, 购买, 特别, 很漂亮, 一只, 小兔子, 推荐, 音乐, 放入, 盒内, 广告宣传, 分包, 包装纸, 时式, 撕扯, 官旗, 强太多, 长度, 长些, 不行, 厉害, 女朋友,

 材质, 好看, 超级, 很小, 两个, 头发, 儿童节, 这是, 好好, 最棒, 简单, 软绵绵, 兔纸, 小尾巴, 飞机, 价钱, 效果, 这款, 精致, 商场, 适中, 价格, 轮子, 不好, 过程, 东西, 犹豫, 转弯, 结构, 模具,

 拼图, 真的, 地方, 子弹, 详细, 完美, 遗憾, 耳朵, 连接, 还会, 轨道, 设计, 精美, 下次, 个头, 购买, 兔子, 熟练, 桌游, 引导, 东东, 电话, 很难, 小小, 支持, 后悔, 是因为, 纸板, 拼出来, 官方,

 宝宝, 满意, 包装, 喜欢, 女儿, 宝贝, 儿子, 价格, 爱不释手, 睡觉, 商品, 不好, 一点, 送人, 我家, 购物, 不错, 性价比, 能力, 锻炼, 有趣, 瑕疵, 海外, 期待, 不太好, 体验, 精细, 愉快, 耐脏, 一岁,

 亚马逊, 小朋友, 可爱, 积分, 特别, 东西, 喜欢, 不用, 评价, 评论, 宝宝, 产品, 推荐, 电池, 值得, 中国, 合适, 好好, 浪费, 发出, 真的, 只能, 既能, 复制到, 复制, 这段话, 换钱, 走到, 省事, 看着

In [51]:
order_centroids

array([[ 306, 1433, 4726, ..., 2191, 2364, 3882],
       [5193, 4818, 1472, ...,   73, 2590, 3595],
       [1903, 1433, 1929, ..., 3940, 5350,  170],
       ..., 
       [ 507, 2045, 1283, ..., 3565,  713, 2590],
       [ 306, 5273, 2841, ..., 1822, 1193, 3449],
       [1729, 2456, 1874, ..., 1732,  713, 3876]])

In [52]:
ftf = transformer.fit_transform(order_centroids)

In [53]:
ftf = ftf.toarray()

In [54]:
#词频矩阵
tfidMMframe = pd.DataFrame(np.round(ftf, 2))
tfidMMframe[:5]

,0,1,2,3,4,5,6,7,8,9,...,5350,5351,5352,5353,5354,5355,5356,5357,5358,5359
0,0.00,0.01,0.02,0.00,0.02,0.01,0.01,0.01,0.00,0.00,...,0.02,0.01,0.00,0.01,0.01,0.02,0.02,0.01,0.01,0.02
1,0.02,0.02,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,...,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.02
2,0.01,0.01,0.01,0.00,0.00,0.02,0.01,0.01,0.02,0.02,...,0.01,0.01,0.00,0.01,0.02,0.02,0.02,0.02,0.02,0.00
3,0.02,0.01,0.02,0.01,0.01,0.02,0.00,0.02,0.02,0.02,...,0.02,0.01,0.02,0.02,0.02,0.02,0.02,0.00,0.02,0.00
4,0.01,0.01,0.02,0.01,0.00,0.01,0.00,0.02,0.01,0.01,...,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01,0.02,0.00


In [55]:
from __future__ import print_function
 
order_centroids =  lda.components_.argsort()[:, ::-1] 
# print(order_centroids)

for i in range(num_clusters):
    print("Cluster %d:" % i, end='\n')
    time.sleep(1)
    for ind in order_centroids[i, :30]: # 每个聚类选 30 个词
        print(' %s' % word[ind], end=',')
        print(' %f' % lda.components_[i][ind-1], end=',')
        print(' %f' % (tfidMMframe[i:i+1][ind]), end=',')#a词在所属群中出现的频率
        print(' %f' % max(tfidframe[:][ind]), end=',') #a词与所有顾客评论中出现最高词频的相对指数
        print(' %f' % (tfidMMframe[i:i+1][ind]/max(tfidframe[:][ind])), end='\n')
        
    print('') # 空行
    print() # 空行

Cluster 0:
 不错, 0.100002, 0.020000, 1.000000, 0.020000
 喜欢, 0.100017, 0.020000, 1.000000, 0.020000
 质量, 0.100005, 0.010000, 1.000000, 0.010000
 做工, 1.261614, 0.020000, 1.000000, 0.020000
 玩具, 0.100009, 0.010000, 0.690000, 0.014493
 孩子, 0.100004, 0.020000, 0.770000, 0.025974
 宝宝, 0.100000, 0.010000, 0.800000, 0.012500
 小孩, 0.100007, 0.020000, 1.000000, 0.020000
 价格, 0.100000, 0.000000, 0.800000, 0.000000
 儿子, 0.100010, 0.000000, 0.860000, 0.000000
 味道, 0.100000, 0.020000, 1.000000, 0.020000
 好玩, 0.100010, 0.010000, 0.880000, 0.011364
 手感, 0.100003, 0.010000, 0.890000, 0.011236
 设计, 0.100014, 0.000000, 0.770000, 0.000000
 图片, 0.100005, 0.010000, 1.000000, 0.010000
 东西, 0.100000, 0.010000, 0.690000, 0.014493
 活动, 0.100017, 0.010000, 0.570000, 0.017544
 异味, 0.100003, 0.010000, 0.760000, 0.013158
 开心, 3.735057, 0.000000, 1.000000, 0.000000
 精致, 0.100022, 0.020000, 0.870000, 0.022989
 小车, 0.100050, 0.020000, 0.520000, 0.038462
 爱玩, 0.100006, 0.000000, 0.790000, 0.000000
 牌子, 0.100002, 0.0200

 亚马逊, 0.100019, 0.010000, 0.580000, 0.017241
 小朋友, 0.100005, 0.020000, 0.860000, 0.023256
 可爱, 0.100000, 0.000000, 1.000000, 0.000000
 积分, 0.100000, 0.020000, 0.640000, 0.031250
 特别, 0.100004, 0.010000, 0.970000, 0.010309
 东西, 0.100000, 0.010000, 0.690000, 0.014493
 喜欢, 0.100024, 0.000000, 1.000000, 0.000000
 不用, 0.100037, 0.000000, 0.430000, 0.000000
 评价, 0.100000, 0.010000, 0.460000, 0.021739
 评论, 0.100027, 0.000000, 0.760000, 0.000000
 宝宝, 0.100003, 0.000000, 0.800000, 0.000000
 产品, 0.100023, 0.010000, 0.750000, 0.013333
 推荐, 0.100012, 0.000000, 0.660000, 0.000000
 电池, 0.100013, 0.020000, 0.570000, 0.035088
 值得, 0.100000, 0.010000, 0.760000, 0.013158
 中国, 0.100003, 0.020000, 0.870000, 0.022989
 合适, 2.082742, 0.020000, 0.750000, 0.026667
 好好, 0.100002, 0.000000, 0.890000, 0.000000
 浪费, 0.100000, 0.020000, 0.330000, 0.060606
 发出, 0.100000, 0.020000, 0.770000, 0.025974
 真的, 0.990067, 0.000000, 0.610000, 0.000000
 只能, 0.100003, 0.010000, 0.600000, 0.016667
 既能, 0.100002, 0.010000, 0.210

In [56]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, vectorizer)

/usr/local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      20.583109        1       1 -0.250624 -0.057981
7      15.310618        1       2 -0.073952  0.227755
8      12.289928        1       3 -0.088882 -0.139544
2      11.863946        1       4 -0.177105  0.084701
6      10.670002        1       5 -0.104063 -0.020879
9       7.399986        1       6  0.077133 -0.195164
3       6.444813        1       7  0.169906  0.144274
4       5.731755        1       8  0.128778 -0.004287
5       5.374650        1       9  0.157268 -0.025516
1       4.331193        1      10  0.161540 -0.013358, topic_info=     Category         Freq Term        Total  loglift  logprob
term                                                          
1903  Default   689.000000   孩子   689.000000  30.0000  30.0000
1433  Default  1133.000000   喜欢  1133.000000  29.0000  29.0000
306   Default   738.000000   不错   738.000000  28.0000  28.0000
4726  Default   521.000000   质量   521.000000  27.0000  27.0000
1929  Default   561.000000   宝宝   561.000000  26.0000  26.0000
707   Default   346.000000   做工   346.000000  25.0000  25.0000
3698  Default   287.000000   玩具   287.000000  24.0000  24.0000
2647  Default    97.000000   拼图    97.000000  23.0000  23.0000
2045  Default   150.000000  小朋友   150.000000  22.0000  22.0000
507   Default   242.000000  亚马逊   242.000000  21.0000  21.0000
3547  Default   114.000000   满意   114.000000  20.0000  20.0000
1067  Default   183.000000   包装   183.000000  19.0000  19.0000
2456  Default   146.000000   感觉   146.000000  18.0000  18.0000
5002  Default   136.000000   适合   136.000000  17.0000  17.0000
1283  Default   186.000000   可爱   186.000000  16.0000  16.0000
1729  Default    89.000000   太小    89.000000  15.0000  15.0000
4033  Default   120.000000   积分   120.000000  14.0000  14.0000
3668  Default   138.000000   特别   138.000000  13.0000  13.0000
3183  Default    74.000000   柔软    74.000000  12.0000  12.0000
730   Default   158.000000   儿子   158.000000  11.0000  11.0000
54    Default   129.000000   一岁   129.000000  10.0000  10.0000
4773  Default    84.000000   超级    84.000000   9.0000   9.0000
570   Default   168.000000   价格   168.000000   8.0000   8.0000
277   Default    97.000000   不用    97.000000   7.0000   7.0000
4650  Default    96.000000   评价    96.000000   6.0000   6.0000
1798  Default    77.000000   女儿    77.000000   5.0000   5.0000
652   Default    99.000000   便宜    99.000000   4.0000   4.0000
3938  Default   101.000000   真的   101.000000   3.0000   3.0000
3139  Default    62.000000   材质    62.000000   2.0000   2.0000
4141  Default    91.000000   简单    91.000000   1.0000   1.0000
...       ...          ...  ...          ...      ...      ...
2436  Topic10     9.446596   想要    10.210375   3.0616  -5.0654
1139  Topic10     9.104200   卡扣     9.868046   3.0588  -5.1024
4776  Topic10     8.736244   超过     9.500167   3.0555  -5.1436
2422  Topic10     8.566699   情况     9.330573   3.0539  -5.1632
1398  Topic10     8.327396   哇哇     9.091144   3.0516  -5.1915
899   Topic10     8.126949   几种     8.891068   3.0495  -5.2159
1756  Topic10     7.363009   太软     8.126819   3.0406  -5.3146
1104  Topic10     7.340944  半小时     8.104698   3.0404  -5.3176
3779  Topic10     6.574491   用时     7.338243   3.0294  -5.4279
1727  Topic10     6.383510  太好了     7.147266   3.0263  -5.4574
4254  Topic10     6.301502   绒毛     7.065254   3.0249  -5.4703
4041  Topic10     6.301502   稀少     7.065254   3.0249  -5.4703
188   Topic10     6.301502   上深     7.065254   3.0249  -5.4703
4204  Topic10     6.214983   紫霞     6.978732   3.0234  -5.4841
4234  Topic10     5.957895   组合     6.721712   3.0187  -5.5264
3110  Topic10     5.950459   木屑     6.714522   3.0185  -5.5276
531   Topic10     5.836674   人员     6.600425   3.0164  -5.5469
1295  Topic10     5.809654   吃惊     6.573524   3.0158  -5.5516
2768  Topic10     5.747387   提供     6.511146   3.0146  -5.5624
3793  Topic10     5.694216   电影  

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-23 00:14:13.954383. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [57]:
# #挑選特定條件(抽樣)
# samples = ldaClusterframe[ldaClusterframe.loc[:,'ClusterLabel'] == 4][:441]
# Rsamples = samples['review'].values
# Rsamples

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-22 15:10:55.964269. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [58]:
# with open('../test.csv','w',encoding='utf-8') as f :
#     f.write(str(Rsamples))

/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2018-02-22 15:10:55.971174. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
